In [1]:
from rasa.core.agent import Agent, load_agent
from rasa.core.policies.ensemble import PolicyEnsemble

prefix='/pi/ws/knowledgebasebot'
model_path='./models'

ensemble_dict = {
        "policies": [
            {
                "epochs": 50,
                "name": "KerasPolicy",
            },
            {"max_history": 5, "priority": 3, "name": "MemoizationPolicy"},
        ]}
ensemble = PolicyEnsemble.from_dict(ensemble_dict)
agent = Agent(
        f"{prefix}/domain.yml",
        policies=ensemble,
    )

training_data = await agent.load_data(f"{prefix}/data/stories.md")
agent.train(training_data)
agent.persist(model_path)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.






INFO:tensorflow:Entry Point [tensor2tensor.envs.tic_tac_toe_env:TicTacToeEnv] registered with id [T2TEnv-TicTacToeEnv-v0]




Processed trackers: 100%|██████████| 321/321 [00:09<00:00, 34.71it/s, # actions=169]



Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 24)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7296      
_________________________________________________________________
dense (Dense)                (None, 13)                429       
_________________________________________________________________
activation (Activation)      (None, 13)                0         
Total params: 7,725
Trainable params: 7,725
Non-trainable params: 0
_________________________________________________________________
Train on 169 samples
Epoch 1/50
169/169 [==============================] - 1

Processed trackers: 100%|██████████| 6/6 [00:00<00:00, 128.35it/s, # actions=17]
Processed actions: 17it [00:00, 614.14it/s, # examples=17]


In [4]:
from rasa.core.channels.channel import UserMessage

# await agent.handle_text("hello")
text = '/greet{"name":"Rasa"}'
message = UserMessage(text, sender_id="test_agent_handle_message")
result = await agent.handle_message(message)
print(result)

[{'recipient_id': 'test_agent_handle_message', 'text': 'Hey!'}]


In [5]:
from rasa.train import train_async
await train_async(
        domain=f"{prefix}/domain.yml",
        config=f"{prefix}/config.yml",
        training_files=f"{prefix}/data/",
        output_path='./models_kb',
    )

Processed Story Blocks:   0%|          | 0/6 [00:00<?, ?it/s, # trackers=26]

Training Core model...


Processed trackers: 100%|██████████| 6/6 [00:00<00:00, 51.10it/s, # actions=17]
Processed actions: 17it [00:00, 207.27it/s, # examples=17]
Processed trackers: 100%|██████████| 321/321 [00:09<00:00, 33.46it/s, # actions=169]


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 24)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7296      
_________________________________________________________________
dense (Dense)                (None, 13)                429       
_________________________________________________________________
activation (Activation)      (None, 13)                0         
Total params: 7,725
Trainable params: 7,725
Non-trainable params: 0
_________________________________________________________________
Train on 169 samples
Epoch 1/100
169/169 [==============================] - 2s 10ms/sample - loss: 2.4605 - acc: 0.4024
Epoch 2/100
169/169 [==============================] - 0s 481us/sample - loss: 2.3265 - acc: 0.5030
Epoch 3/100
169/169 [===========

/Users/xiaofeiwu/miniconda3/envs/rasa/lib/python3.6/site-packages/rasa/nlu/training_data/training_data.py:340: UserWarning: Entity 'breakfast-included' has only 1 training examples! minimum is 2, training may fail.
  "".format(entity_type, count, self.MIN_EXAMPLES_PER_ENTITY)


Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.






Epochs: 100%|██████████| 300/300 [00:06<00:00, 45.72it/s, loss=0.453, acc=1.000]
/Users/xiaofeiwu/miniconda3/envs/rasa/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)



NLU model training completed.
Your Rasa model is trained and saved at '/Users/xiaofeiwu/jcloud/assets/langs/workspace/rasa/rasa-1.x/ws/sagas-ai/models_kb/20191129-011049.tar.gz'.


'./models_kb/20191129-011049.tar.gz'

In [10]:
agent_kb = Agent.load('./models_kb/20191129-011049.tar.gz')
await agent_kb.handle_text("hello")

INFO:tensorflow:Restoring parameters from /var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmpwi2m1t6n/nlu/component_6_EmbeddingIntentClassifier.ckpt


[{'recipient_id': 'default', 'text': 'Hello! How can I help you?'}]

In [11]:
# agent_kb.update_model(
#         None, None, agent.fingerprint, agent.interpreter, agent.model_directory
#     )

In [12]:
await agent_kb.handle_text("hello")

[{'recipient_id': 'default', 'text': 'Hey!'}]

In [15]:
from rasa.core.trackers import DialogueStateTracker
from rasa.core.slots import Slot
from pprint import pprint

tracker = DialogueStateTracker.from_dict("1", [], [Slot("requested_language")])
# we'll expect this value 'en' to be part of the result from the interpreter
tracker._set_slot("requested_language", "en")
result = await agent_kb.parse_message_using_nlu_interpreter("lunch?", tracker)
pprint(result)

{'entities': [],
 'intent': {'confidence': 0.3773570954799652, 'name': 'query_knowledge_base'},
 'intent_ranking': [{'confidence': 0.3773570954799652,
                     'name': 'query_knowledge_base'},
                    {'confidence': 0.3278685212135315, 'name': 'goodbye'},
                    {'confidence': 0.1596766710281372, 'name': 'greet'},
                    {'confidence': 0.13509775698184967,
                     'name': 'bot_challenge'}],
 'text': 'lunch?'}


In [16]:
sender_id='default'
tracker = agent_kb.tracker_store.get_or_create_tracker(sender_id)
print(";".join([e.as_story_string() for e in tracker.events if e.as_story_string()]))

action_listen;greet;utter_greet;action_listen;greet;utter_greet;action_listen


In [5]:
# import logging
# import rasa.utils.io as io_utils
# from rasa.core.validator import Validator

# logger = logging.getLogger(__name__)

# io_utils.configure_colored_logging('DEBUG')

# proj_prefix='/pi/ws/simple'
# validator = Validator.from_files(domain_file=f'{proj_prefix}/domain.yml',
#                                  nlu_data=f'{proj_prefix}/data/nlu_data.md',
#                                  stories=f'{proj_prefix}/data/stories.md')

# validator.verify_all()

## see: tests/core/test_validator.py